## Preprocessing financial statements using Gemini

In this notebook, I preprocess the downloaded reports as far as required in order to feed them into the LLM to obtain a buy/sell recommendation.
Reuters uses abbreviations for all positions in financial statements. Using Gemini to infer the full position name considerably helps the process of preparing the dataset.

In [8]:
import pandas as pd
import numpy as np 
from google import genai
from google.genai import types
import json
import re
from tqdm import tqdm
import time
from google.genai.errors import ServerError  

In [9]:
# Import Gemini API key
with open("../proton_google_api_key.txt", "r") as f:
    key = f.read().strip()    

# Initialize the Gemini client with the API key
client = genai.Client(api_key = key)

In [ ]:
# Read in company names to look up buy/sell recommendations for
sp400_companies = pd.read_csv("../data/sp400_companies.csv", dtype={"CIK": str})
sp500_companies = pd.read_csv("../data/sp500_companies.csv", dtype={"CIK": str})
sp600_companies = pd.read_csv("../data/sp600_companies.csv", dtype={"CIK": str})

# Combine all CIKs into a single list
sp400_ciks = sp400_companies["CIK"].tolist()
sp500_ciks = sp500_companies["CIK"].tolist()
sp600_ciks = sp600_companies["CIK"].tolist()
ciks = sp400_ciks + sp500_ciks + sp600_ciks

# Read in RICs to match with CIKs
rics = pd.read_csv("../data/rics.csv", dtype={"Instrument": str})
# Ranem Instrument to CIK for easier merging
rics.rename(columns={"Instrument": "CIK"}, inplace=True)

- Import and further process balance sheet df

In [ ]:
first_balance_sheets = pd.read_csv("../data/balance_sheets.csv", dtype={"CIK": str})
missing_balance_sheets = pd.read_csv("../data/missing_balance_sheets.csv", dtype={"CIK": str})
balance_sheets = pd.concat([first_balance_sheets, missing_balance_sheets], ignore_index=True)
# Delete entries where STD Balance Sheet All and FCC Item Name are missing
balance_sheets = balance_sheets.dropna(subset=["STD Balance Sheet All", "FCC Item Name"])
balance_sheets.head()

,Date,STD Balance Sheet All,FCC Item Name,RIC,Statement
4,2016-06-30,332000000.0,TR.F.CashSTInvst,AA.N,balance_sheet
5,2016-06-30,332000000.0,TR.F.CashCashEquiv,AA.N,balance_sheet
6,2016-06-30,74000000.0,TR.F.DerivFinInstrHedgeST,AA.N,balance_sheet
7,2016-06-30,607000000.0,TR.F.LoansRcvblNetST,AA.N,balance_sheet
8,2016-06-30,426000000.0,TR.F.TradeAcctTradeNotesRcvblNet,AA.N,balance_sheet


In [5]:
# Adjust Item Name to give a clearer indication of the position
unique_balance_sheet_positions = balance_sheets["FCC Item Name"].unique()

In this cell, I employ Gemini to return actual official balance sheet position labels for the Thomson Reuters abbreviations contained in financial statements downloaded via the LSEG Data Library for Python.

In [ ]:
balance_sheet_labels = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        temperature=0.1,
        system_instruction="You are a financial analyst, that provides concise and accurate answers.",
        thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking
    ),
    contents=[f"""The following is a list of abbreviations used by Thomsom Reuters in their reporting database. For every given abbreviation,
              please provide the actual financial position this refers to in a balance sheet. Example: TR.F.TotAssets: Total Assets.
              Here are the abbreviations: {unique_balance_sheet_positions.tolist()} Provide your answer in a dictionary style format.
              """],  
)

In [ ]:
# Regex to extract JSON from the response
# First I extract the JSON string from the response text
json_str = re.search(r'{.*}', balance_sheet_labels.text, re.DOTALL).group()
# Then I parse the JSON string into a Python dictionary
data = json.loads(json_str)
# Save the dictionary to a file
with open("../data/balance_sheet_labels.json", "w") as f:
    json.dump(data, f, indent=4)  # indent makes JSON-format more readable

In [16]:
# Finally, this dictionary can be used to map the abbreviations to the actual financial positions in the balance sheets.
balance_sheets["position_label"] = balance_sheets["FCC Item Name"].map(data)

In [17]:
# Merge on RIC to include CIK and RIC in the balance sheets
balance_sheets = balance_sheets.merge(rics, left_on="RIC", right_on="RIC", how="left")
balance_sheets.head(10)

,Date,STD Balance Sheet All,FCC Item Name,RIC,Statement,position_label,CIK
0,2016-06-30,3.320000e+08,TR.F.CashSTInvst,AA.N,balance_sheet,Cash and Short-Term Investments,0001675149
1,2016-06-30,3.320000e+08,TR.F.CashCashEquiv,AA.N,balance_sheet,Cash and Cash Equivalents,0001675149
2,2016-06-30,7.400000e+07,TR.F.DerivFinInstrHedgeST,AA.N,balance_sheet,"Derivative Financial Instruments, Hedging, Sho...",0001675149
3,2016-06-30,6.070000e+08,TR.F.LoansRcvblNetST,AA.N,balance_sheet,"Loans and Receivables, Net, Short-Term",0001675149
4,2016-06-30,4.260000e+08,TR.F.TradeAcctTradeNotesRcvblNet,AA.N,balance_sheet,"Trade Accounts and Trade Notes Receivable, Net",0001675149
5,2016-06-30,1.810000e+08,TR.F.RcvblOthTot,AA.N,balance_sheet,"Receivables, Other, Total",0001675149
6,2016-06-30,1.166000e+09,TR.F.InvntTot,AA.N,balance_sheet,"Inventory, Total",0001675149
7,2016-06-30,8.350000e+08,TR.F.InvntRawMaterial,AA.N,balance_sheet,"Inventory, Raw Material",0001675149
8,2016-06-30,1.760000e+08,TR.F.InvntWorkInProg,AA.N,balance_sheet,"Inventory, Work In Progress",0001675149
9,2016-06-30,1.550000e+08,TR.F.InvntFinGoods,AA.N,balance_sheet,"Inventory, Finished Goods",0001675149


In [ ]:
# Save to CSV
balance_sheets.to_csv("../data/balance_sheets_with_labels.csv", index=False)

---
- Import and further process Cash Flow statement df

In [ ]:
first_cash_flow_statements = pd.read_csv("../data/cash_flow_statements.csv", dtype={"CIK": str})
missing_cash_flow_statements = pd.read_csv("../data/missing_cash_flow_statements.csv", dtype={"CIK": str})
cash_flow_statements = pd.concat([first_cash_flow_statements, missing_cash_flow_statements], ignore_index=True)
cash_flow_statements.head()

,Date,STD Cash Flow All,FCC Item Name,RIC,statement,Statement
0,2011-09-30,NaN,NaN,AA.N,cashflow,NaN
1,2012-09-30,NaN,NaN,AA.N,cashflow,NaN
2,2013-09-30,NaN,NaN,AA.N,cashflow,NaN
3,2014-09-30,NaN,NaN,AA.N,cashflow,NaN
4,2016-06-30,-19000000.0,TR.F.ProfLossStartingLineCF,AA.N,cashflow,NaN


In [5]:
# Fix wrong column names
cash_flow_statements = cash_flow_statements.drop(columns = "Statement")
cash_flow_statements.rename(columns={"statement": "Statement"}, inplace=True)
# Delete entries where both STD Cash Flow All and FCC Item Name is NaN
cash_flow_statements = cash_flow_statements.dropna(subset=["STD Cash Flow All", "FCC Item Name"], how="all")
cash_flow_statements.head()

,Date,STD Cash Flow All,FCC Item Name,RIC,Statement
4,2016-06-30,-19000000.0,TR.F.ProfLossStartingLineCF,AA.N,cashflow
5,2016-06-30,245000000.0,TR.F.NonCashItemsReconcAdjCF,AA.N,cashflow
6,2016-06-30,16000000.0,TR.F.EqIncLossInNetEarnCF,AA.N,cashflow
7,2016-06-30,103000000.0,TR.F.OthNonCashItemsReconcAdjCF,AA.N,cashflow
8,2016-06-30,178000000.0,TR.F.DeprDeplAmortInclImpairCF,AA.N,cashflow


In [6]:
# Get unique cash flow positions
unique_cash_flow_positions = cash_flow_statements["FCC Item Name"].unique()

- As before, Gemini is employed to infer the correct cash flow labels from the FCC Item Name Codes

In [9]:
cash_flow_labels = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        temperature=0.1,
        system_instruction="You are a financial analyst, that provides concise and accurate answers.",
        thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking
    ),
    contents=[f"""The following is a list of abbreviations used by Thomsom Reuters in their reporting database. For every given abbreviation,
              please provide the actual financial position this refers to in a cash flow statement. Example: TR.F.TotAssets: Total Assets.
              Here are the abbreviations: {unique_cash_flow_positions.tolist()} Provide your answer in a dictionary style format.
              """],  
)

In [10]:
# Regex to extract JSON from the response
# First I extract the JSON string from the response text
json_str = re.search(r'{.*}', cash_flow_labels.text, re.DOTALL).group()
# Then I parse the JSON string into a Python dictionary
data = json.loads(json_str)
# Save the dictionary to a file
with open("data/cash_flow_labels.json", "w") as f:
    json.dump(data, f, indent=4)  

In [11]:
# Map dictionary to cash flow statements
cash_flow_statements["position_label"] = cash_flow_statements["FCC Item Name"].map(data)

In [12]:
# Merge on RIC to include CIK and RIC in the cash flow statements
cash_flow_statements = cash_flow_statements.merge(rics, left_on="RIC", right_on="RIC", how="left")
cash_flow_statements.head(10)

,Date,STD Cash Flow All,FCC Item Name,RIC,Statement,position_label,CIK
0,2016-06-30,-19000000.0,TR.F.ProfLossStartingLineCF,AA.N,cashflow,Profit/Loss Starting Line (Cash Flow),0001675149
1,2016-06-30,245000000.0,TR.F.NonCashItemsReconcAdjCF,AA.N,cashflow,Non-Cash Items Reconciliation Adjustments (Cas...,0001675149
2,2016-06-30,16000000.0,TR.F.EqIncLossInNetEarnCF,AA.N,cashflow,Equity Income/Loss in Net Earnings (Cash Flow),0001675149
3,2016-06-30,103000000.0,TR.F.OthNonCashItemsReconcAdjCF,AA.N,cashflow,Other Non-Cash Items Reconciliation Adjustment...,0001675149
4,2016-06-30,178000000.0,TR.F.DeprDeplAmortInclImpairCF,AA.N,cashflow,"Depreciation, Depletion, Amortization, Includi...",0001675149
5,2016-06-30,178000000.0,TR.F.DeprDeplPPECF,AA.N,cashflow,"Depreciation, Depletion of Property, Plant, an...",0001675149
6,2016-06-30,-28000000.0,TR.F.DefIncTaxIncTaxCreditsCF,AA.N,cashflow,Deferred Income Tax and Income Tax Credits (Ca...,0001675149
7,2016-06-30,-34000000.0,TR.F.AssetsSaleGLCF,AA.N,cashflow,Assets Sale Gain/Loss (Cash Flow),0001675149
8,2016-06-30,10000000.0,TR.F.ShrBasedPaymtCF,AA.N,cashflow,Share-Based Payment (Cash Flow),0001675149
9,2016-06-30,226000000.0,TR.F.CashFlowOpBefChgInWkgCap,AA.N,cashflow,Cash Flow from Operations Before Changes in Wo...,0001675149


In [ ]:
# Save to CSV
cash_flow_statements.to_csv("../data/cash_flow_statements_with_labels.csv", index=False)

---
- Import and further process income statements

In [ ]:
first_income_statements = pd.read_csv("../data/income_statements.csv", dtype={"CIK": str})
missing_income_statements = pd.read_csv("../data/missing_income_statements.csv", dtype={"CIK": str})
income_statements = pd.concat([first_income_statements, missing_income_statements], ignore_index=True)
income_statements.head()

,Date,STD Income Statement All,FCC Item Name,RIC,statement,Statement
0,2011-09-30,NaN,NaN,AA.N,income_statement,NaN
1,2012-09-30,NaN,NaN,AA.N,income_statement,NaN
2,2013-09-30,NaN,NaN,AA.N,income_statement,NaN
3,2014-09-30,NaN,NaN,AA.N,income_statement,NaN
4,2016-06-30,2.323000e+09,TR.F.RevGoodsSrvc,AA.N,income_statement,NaN


In [23]:
# Drop observations, where both STD Income Statement All and FCC Item Name is NaN
income_statements = income_statements.dropna(subset=["STD Income Statement All", "FCC Item Name"], how="all")
# Get rid of Statement column, which is result of wrong spelling
income_statements = income_statements.drop(columns = "Statement")

In [17]:
# Unique positions in income statements
unique_income_positions = income_statements["FCC Item Name"].unique()

- One last time, Gemini is employed to map the FCC Item Name Codes to more official labels used in Cash Flow statements

In [18]:
income_statement_labels = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        temperature=0.1,
        system_instruction="You are a financial analyst, that provides concise and accurate answers.",
        thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking
    ),
    contents=[f"""The following is a list of abbreviations used by Thomsom Reuters in their reporting database. For every given abbreviation,
              please provide the actual financial position this refers to in an income statement. Example: TR.F.TotAssets: Total Assets.
              Here are the abbreviations: {unique_income_positions.tolist()} Provide your answer in a dictionary style format.
              """],  
)

In [ ]:
# Regex to extract JSON from the response
# First I extract the JSON string from the response text
json_str = re.search(r'{.*}', income_statement_labels.text, re.DOTALL).group()
# Then I parse the JSON string into a Python dictionary
data = json.loads(json_str)
# Save the dictionary to a file
with open("../data/income_statement_labels.json", "w") as f:   
    json.dump(data, f, indent=4)

In [24]:
# Map dictionary to cash flow statements
income_statements["position_label"] = income_statements["FCC Item Name"].map(data)
income_statements.head()

,Date,STD Income Statement All,FCC Item Name,RIC,statement,position_label
4,2016-06-30,2.323000e+09,TR.F.RevGoodsSrvc,AA.N,income_statement,Revenue from Goods and Services
5,2016-06-30,2.323000e+09,TR.F.SalesOfGoodsSrvcNetUnclassif,AA.N,income_statement,"Sales of Goods and Services, Net, Unclassified"
6,2016-06-30,2.323000e+09,TR.F.TotRevenue,AA.N,income_statement,Total Revenue
7,2016-06-30,2.119000e+09,TR.F.CostOfOpRev,AA.N,income_statement,Cost of Operating Revenue
8,2016-06-30,2.119000e+09,TR.F.COGSTot,AA.N,income_statement,"Cost of Goods Sold, Total"


In [25]:
# Merge on RIC to include CIK and RIC in the cash flow statements
income_statements = income_statements.merge(rics, left_on="RIC", right_on="RIC", how="left")
income_statements.head(10)

,Date,STD Income Statement All,FCC Item Name,RIC,statement,position_label,CIK
0,2016-06-30,2.323000e+09,TR.F.RevGoodsSrvc,AA.N,income_statement,Revenue from Goods and Services,0001675149
1,2016-06-30,2.323000e+09,TR.F.SalesOfGoodsSrvcNetUnclassif,AA.N,income_statement,"Sales of Goods and Services, Net, Unclassified",0001675149
2,2016-06-30,2.323000e+09,TR.F.TotRevenue,AA.N,income_statement,Total Revenue,0001675149
3,2016-06-30,2.119000e+09,TR.F.CostOfOpRev,AA.N,income_statement,Cost of Operating Revenue,0001675149
4,2016-06-30,2.119000e+09,TR.F.COGSTot,AA.N,income_statement,"Cost of Goods Sold, Total",0001675149
5,2016-06-30,1.941000e+09,TR.F.COGSUnclassif,AA.N,income_statement,"Cost of Goods Sold, Unclassified",0001675149
6,2016-06-30,1.780000e+08,TR.F.DeprInCOGS,AA.N,income_statement,Depreciation in Cost of Goods Sold,0001675149
7,2016-06-30,2.040000e+08,TR.F.GrossProfIndPropTot,AA.N,income_statement,"Gross Profit, Industrial Property, Total",0001675149
8,2016-06-30,9.700000e+07,TR.F.SGATot,AA.N,income_statement,"Selling, General and Administrative Expenses, ...",0001675149
9,2016-06-30,9.000000e+07,TR.F.SGAUnclassif,AA.N,income_statement,"Selling, General and Administrative Expenses, ...",0001675149


In [ ]:
# Save to CSV
income_statements.to_csv("../data/income_statements_with_labels.csv", index=False)

---

- Function to fetch most recent company filings for a given CIK and Date

In [4]:
labeled_balance_sheets = pd.read_csv("../data/balance_sheets_with_labels.csv", dtype={"CIK": str})
labeled_income_statements = pd.read_csv("../data/income_statements_with_labels.csv", dtype={"CIK": str})
labeled_cash_flow_statements = pd.read_csv("../data/cash_flow_statements_with_labels.csv", dtype={"CIK": str})

C:\Users\benny\AppData\Local\Temp\ipykernel_22576\2672417944.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  labeled_income_statements = pd.read_csv("../data/income_statements_with_labels.csv", dtype={"CIK": str})
C:\Users\benny\AppData\Local\Temp\ipykernel_22576\2672417944.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  labeled_cash_flow_statements = pd.read_csv("../data/cash_flow_statements_with_labels.csv", dtype={"CIK": str})


In [5]:
bs_uniques = labeled_balance_sheets["CIK"].unique()
is_uniques = labeled_income_statements["CIK"].unique()
cs_uniques = labeled_cash_flow_statements["CIK"].unique()

# Find overlap 
overlap = set(bs_uniques) & set(is_uniques) & set(cs_uniques)
len(overlap)

1490

In [37]:
def get_llm_ratings(cik: str, balance_sheets = None, income_statements = None, cash_flow_statements = None):
    
    """
    Function that returns a DataFrame with LLM ratings for a given CIK.
    For every reporting date, the function fetches the most recent financial statements, i.e.
    - Balance Sheet
    - Cash Flow Statement
    - Income Statement
    and calls the LLM to get a buy/sell/hold recommendation. In order to avoid issues with reports that were filed slightly apart,
    a window of 10 days around a given reporting date is used. This helps to ensure that the LLM has access to all relevant financial information for a given reporting date.
    Furthermore, reports that were filed slightly apart will not lead to recommendations that are based on partial information only and will also not cause multiple
    recommendations that only lie within the window of 10 days around a given reporting date.
    
    Parameters: 
    cik: str, CIK of the company (Can be looked up on the SEC website)
    balance_sheets: Balance Sheets DataFrame with columns: STD Balance Sheet All, FCC Item Name, CIK and Date (among others)
    income_statements: Income Statements DataFrame with columns: STD Income Statement All, FCC Item Name, CIK and Date (among others)
    cash_flow_statements: Cash Flow Statements DataFrame with columns: STD Cash Flow All, FCC Item Name, CIK and Date (among others)
    """
    
    # First filter dfs for input CIK
    balance_sheets = balance_sheets[balance_sheets["CIK"] == cik].copy()
    income_statements = income_statements[income_statements["CIK"] == cik].copy()
    cash_flow_statements = cash_flow_statements[cash_flow_statements["CIK"] == cik].copy()
    
    # Convert the date columns to datetime objects
    for df in [balance_sheets, income_statements, cash_flow_statements]:
        df["Report Date"] = pd.to_datetime(df["Date"])

    # Determine unique dates
    reporting_dates = pd.concat([
        balance_sheets["Report Date"],
        income_statements["Report Date"],
        cash_flow_statements["Report Date"]
    ]).unique()

    # Sort dates just to be safe
    reporting_dates = np.sort(reporting_dates)

    # In order to handle reports, that were filed slighty apart, a window of 10 days around a given reporting date is used
    window = pd.Timedelta(days=10)

    # Loop over reporting dates to obtain LLM ratings
    llm_ratings = []
    for date in reporting_dates:

        # Subset all financial statements for the given dates +- window days
        # Current quarter
        bs = balance_sheets[(balance_sheets["Report Date"] >= date - window) & (balance_sheets["Report Date"] <= date + window)]
        is_ = income_statements[(income_statements["Report Date"] >= date - window) & (income_statements["Report Date"] <= date + window)]
        cf = cash_flow_statements[(cash_flow_statements["Report Date"] >= date - window) & (cash_flow_statements["Report Date"] <= date + window)]

        # If any of the DataFrames is empty, skip this date and indicate which df was missing
        if bs.empty or is_.empty or cf.empty:
            missing = []
            if bs.empty:
                missing.append("Balance Sheet")
            if is_.empty:
                missing.append("Income Statement")
            if cf.empty:
                missing.append("Cash Flow Statement")
            print(f"Reports missing for date {date}: {', '.join(missing)}. Skipping...")
            continue


        # Concatenate reports into a string with correct labels
        bs_str = "\n".join(bs.apply(lambda row: f"{row['position_label']}: {row['STD Balance Sheet All']}", axis=1).astype(str))
        is_str = "\n".join(is_.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
        cf_str = "\n".join(cf.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))


        # Call the LLM to get the rating
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite", # "gemini-2.5-flash"
            config=types.GenerateContentConfig(
                temperature=0, # Deterministic ouput
                system_instruction="""You are an experienced, data-driven financial analyst, that provides concise and accurate answers.""",
                
                thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking, but only required for Gemini 2.5
            ),
            
            contents=[f"""
            Based on the following financial reports only, please provide an investment recommendation for the underlying company.
                      
            Balance Sheet: 
            {bs_str}

            Income Statement: 
            {is_str}

            Cash Flow Statement: 
            {cf_str}

            Provide your answer using only one of the following signals: 'strong buy', 'buy', 'hold', 'sell', or 'strong sell'.
            """]
        )

        # Extract rating from the response
        rating = response.text.strip().lower()
        llm_ratings.append({
            "cik": str(cik),  # Ensure CIK is a string
            "date": date,
            "rating": rating
        })
    
    # Convert the list of dictionaries to a DataFrame
    llm_ratings_df = pd.DataFrame(llm_ratings)

    # Robustness, since date was missing sometimes (I assume the df was entirely empty..?)
    if llm_ratings_df.empty:
        print(f"No valid ratings found for CIK {cik} on date {date}.")
        return None

    # Convert Report Date to datetime
    llm_ratings_df["date"] = pd.to_datetime(llm_ratings_df["date"]).dt.date

    # Sort by Report Date
    llm_ratings_df.sort_values(by="date", inplace=True)

    # Reset index
    llm_ratings_df.reset_index(drop=True, inplace=True)
    
    return llm_ratings_df

In [ ]:
def create_requests(cik: str, balance_sheets = None, income_statements = None, cash_flow_statements = None):
    
    """
    Function that returns a DataFrame with LLM ratings for a given CIK.
    For every reporting date, the function fetches the most recent financial statements, i.e.
    - Balance Sheet
    - Cash Flow Statement
    - Income Statement
    and calls the LLM to get a buy/sell/hold recommendation. In order to avoid issues with reports that were filed slightly apart,
    a window of 10 days around a given reporting date is used. This helps to ensure that the LLM has access to all relevant financial information for a given reporting date.
    Furthermore, reports that were filed slightly apart will not lead to recommendations that are based on partial information only and will also not cause multiple
    recommendations that only lie within the window of 10 days around a given reporting date.
    
    Parameters: 
    cik: str, CIK of the company (Can be looked up on the SEC website)
    balance_sheets: Balance Sheets DataFrame with columns: STD Balance Sheet All, FCC Item Name, CIK and Date (among others)
    income_statements: Income Statements DataFrame with columns: STD Income Statement All, FCC Item Name, CIK and Date (among others)
    cash_flow_statements: Cash Flow Statements DataFrame with columns: STD Cash Flow All, FCC Item Name, CIK and Date (among others)
    """
    print(f"Creating requests for CIK: {cik}")
    # First filter dfs for input CIK
    balance_sheets = balance_sheets[balance_sheets["CIK"] == cik].copy()
    income_statements = income_statements[income_statements["CIK"] == cik].copy()
    cash_flow_statements = cash_flow_statements[cash_flow_statements["CIK"] == cik].copy()
    
    # Convert the date columns to datetime objects
    for df in [balance_sheets, income_statements, cash_flow_statements]:
        df["Report Date"] = pd.to_datetime(df["Date"])

    # Determine unique dates
    reporting_dates = pd.concat([
        balance_sheets["Report Date"],
        income_statements["Report Date"],
        cash_flow_statements["Report Date"]
    ]).unique()

    # Sort dates just to be safe
    reporting_dates = np.sort(reporting_dates)

    # In order to handle reports, that were filed slighty apart, a window of 10 days around a given reporting date is used
    window = pd.Timedelta(days=10)

    # Loop over reporting dates to obtain LLM ratings
    requests = []
    for i, date in enumerate(reporting_dates):


        # Determine current date
        q0_date = date
        # Determine date of previous quarter
        qminus1_date = date - pd.DateOffset(months=3)
        # Determine date of q0 -2 
        qminus2_date = date - pd.DateOffset(months=6)
        # Determine date of q0 -3
        qminus3_date = date - pd.DateOffset(months=9)
        #print(f"Processing date {i+1}/{len(reporting_dates)}: {date} (Q0: {q0_date}, Q-1: {qminus1_date}, Q-2: {qminus2_date}, Q-3: {qminus3_date})")
        


        # Subset all financial statements for the given dates +- window days
        # Current quarter
        bs = balance_sheets[
            (balance_sheets["Report Date"] >= date - window) &
            (balance_sheets["Report Date"] <= date + window)
        ]
        is_0 = income_statements[
            (income_statements["Report Date"] >= date - window) &
            (income_statements["Report Date"] <= date + window)
        ]
        cf_0 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= date - window) &
            (cash_flow_statements["Report Date"] <= date + window)
        ]

        # Previous quarter (Q-1)
        is_qminus1 = income_statements[
            (income_statements["Report Date"] >= qminus1_date - window) &
            (income_statements["Report Date"] <= qminus1_date + window)
        ]
        cf_qminus1 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus1_date - window) &
            (cash_flow_statements["Report Date"] <= qminus1_date + window)
        ]

        # Two quarters ago (Q-2)
        is_qminus2 = income_statements[
            (income_statements["Report Date"] >= qminus2_date - window) &
            (income_statements["Report Date"] <= qminus2_date + window)
        ]
        cf_qminus2 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus2_date - window) &
            (cash_flow_statements["Report Date"] <= qminus2_date + window)
        ]

        # Three quarters ago (Q-3)
        is_qminus3 = income_statements[
            (income_statements["Report Date"] >= qminus3_date - window) &
            (income_statements["Report Date"] <= qminus3_date + window)
        ]
        cf_qminus3 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus3_date - window) &
            (cash_flow_statements["Report Date"] <= qminus3_date + window)
        ]

        # If no reports are available for the given date, skip to next date
        if bs.empty or is_0.empty or cf_0.empty:
            continue
        
        # Concatenate reports into strings with correct labels
        bs_str = "\n".join(bs.apply(lambda row: f"{row['position_label']}: {row['STD Balance Sheet All']}", axis=1).astype(str))
        is_str = "\n".join(is_0.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
        cf_str = "\n".join(cf_0.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))

        # Append previous quarters if available — even if just one of IS or CF is present
        if not is_qminus1.empty:
            is_qminus1_str = "\n".join(is_qminus1.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from previous quarter:\n{is_qminus1_str}"
        if not cf_qminus1.empty:
            cf_qminus1_str = "\n".join(cf_qminus1.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from previous quarter:\n{cf_qminus1_str}"

        if not is_qminus2.empty:
            is_qminus2_str = "\n".join(is_qminus2.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from two quarters ago:\n{is_qminus2_str}"
        if not cf_qminus2.empty:
            cf_qminus2_str = "\n".join(cf_qminus2.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from two quarters ago:\n{cf_qminus2_str}"

        if not is_qminus3.empty:
            is_qminus3_str = "\n".join(is_qminus3.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from three quarters ago:\n{is_qminus3_str}"
        if not cf_qminus3.empty:
            cf_qminus3_str = "\n".join(cf_qminus3.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from three quarters ago:\n{cf_qminus3_str}"
            
        contents=[f"""
        Based on the following financial reports only, please provide an investment recommendation for the underlying company.
                    
        Balance Sheet: 
        {bs_str}

        Income Statement: 
        {is_str}     

        Cash Flow Statement: 
        {cf_str}

        Provide your answer using only one of the following signals: 'buy', 'hold' or 'sell'.
        """]
        requests.append({
            "cik": str(cik),  # Ensure CIK is a string
            "date": date,
            "contents": contents
        })


    
    return requests

In [ ]:
def get_llm_ratings_with_previous_quarters(cik: str, balance_sheets = None, income_statements = None, cash_flow_statements = None):
    
    """
    Function that returns a DataFrame with LLM ratings for a given CIK.
    For every reporting date, the function fetches the most recent financial statements and those of 3 previous quarters, i.e.
    - Balance Sheet
    - Cash Flow Statement
    - Income Statement
    and calls the LLM to get a buy/sell/hold recommendation. In order to avoid issues with reports that were filed slightly apart,
    a window of 10 days around a given reporting date is used. This helps to ensure that the LLM has access to all relevant financial information for a given reporting date.
    Furthermore, reports that were filed slightly apart will not lead to recommendations that are based on partial information only and will also not cause multiple
    recommendations that only lie within the window of 10 days around a given reporting date.
    
    Parameters: 
    cik: str, CIK of the company (Can be looked up on the SEC website)
    balance_sheets: Balance Sheets DataFrame with columns: STD Balance Sheet All, FCC Item Name, CIK and Date (among others)
    income_statements: Income Statements DataFrame with columns: STD Income Statement All, FCC Item Name, CIK and Date (among others)
    cash_flow_statements: Cash Flow Statements DataFrame with columns: STD Cash Flow All, FCC Item Name, CIK and Date (among others)
    """
    
    # First filter dfs for input CIK
    balance_sheets = balance_sheets[balance_sheets["CIK"] == cik].copy()
    income_statements = income_statements[income_statements["CIK"] == cik].copy()
    cash_flow_statements = cash_flow_statements[cash_flow_statements["CIK"] == cik].copy()
    
    # Convert the date columns to datetime objects
    for df in [balance_sheets, income_statements, cash_flow_statements]:
        df["Report Date"] = pd.to_datetime(df["Date"])

    # Determine unique dates
    reporting_dates = pd.concat([
        balance_sheets["Report Date"],
        income_statements["Report Date"],
        cash_flow_statements["Report Date"]
    ]).unique()

    # Sort dates just to be safe
    reporting_dates = np.sort(reporting_dates)

    # In order to handle reports, that were filed slighty apart, a window of 10 days around a given reporting date is used
    window = pd.Timedelta(days=10)

    # Loop over reporting dates to obtain LLM ratings
    llm_ratings = []
    for i, date in enumerate(reporting_dates):


        # Determine current date
        q0_date = date
        # Determine date of previous quarter
        qminus1_date = date - pd.DateOffset(months=3)
        # Determine date of q0 -2 
        qminus2_date = date - pd.DateOffset(months=6)
        # Determine date of q0 -3
        qminus3_date = date - pd.DateOffset(months=9)        


        # Subset all financial statements for the given dates +- window days
        # Current quarter
        bs = balance_sheets[
            (balance_sheets["Report Date"] >= date - window) &
            (balance_sheets["Report Date"] <= date + window)
        ]
        is_0 = income_statements[
            (income_statements["Report Date"] >= date - window) &
            (income_statements["Report Date"] <= date + window)
        ]
        cf_0 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= date - window) &
            (cash_flow_statements["Report Date"] <= date + window)
        ]

        # Previous quarter (Q-1)
        is_qminus1 = income_statements[
            (income_statements["Report Date"] >= qminus1_date - window) &
            (income_statements["Report Date"] <= qminus1_date + window)
        ]
        cf_qminus1 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus1_date - window) &
            (cash_flow_statements["Report Date"] <= qminus1_date + window)
        ]

        # Two quarters ago (Q-2)
        is_qminus2 = income_statements[
            (income_statements["Report Date"] >= qminus2_date - window) &
            (income_statements["Report Date"] <= qminus2_date + window)
        ]
        cf_qminus2 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus2_date - window) &
            (cash_flow_statements["Report Date"] <= qminus2_date + window)
        ]

        # Three quarters ago (Q-3)
        is_qminus3 = income_statements[
            (income_statements["Report Date"] >= qminus3_date - window) &
            (income_statements["Report Date"] <= qminus3_date + window)
        ]
        cf_qminus3 = cash_flow_statements[
            (cash_flow_statements["Report Date"] >= qminus3_date - window) &
            (cash_flow_statements["Report Date"] <= qminus3_date + window)
        ]

        # If no reports are available for the given date, skip to next date
        if bs.empty or is_0.empty or cf_0.empty:
            missing = []
            if bs.empty:
                missing.append("Balance Sheet")
            if is_0.empty:
                missing.append("Income Statement at Q0")
            if cf_0.empty:
                missing.append("Cash Flow Statement at Q0")
            print(f"Reports missing for date {date}: {', '.join(missing)}. Skipping...")
            continue
        
        # Concatenate reports into strings with correct labels
        bs_str = "\n".join(bs.apply(lambda row: f"{row['position_label']}: {row['STD Balance Sheet All']}", axis=1).astype(str))
        is_str = "\n".join(is_0.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
        cf_str = "\n".join(cf_0.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))

        # Append previous quarters if available — even if just one of IS or CF is present
        if not is_qminus1.empty:
            is_qminus1_str = "\n".join(is_qminus1.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from previous quarter:\n{is_qminus1_str}"
        if not cf_qminus1.empty:
            cf_qminus1_str = "\n".join(cf_qminus1.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from previous quarter:\n{cf_qminus1_str}"

        if not is_qminus2.empty:
            is_qminus2_str = "\n".join(is_qminus2.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from two quarters ago:\n{is_qminus2_str}"
        if not cf_qminus2.empty:
            cf_qminus2_str = "\n".join(cf_qminus2.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from two quarters ago:\n{cf_qminus2_str}"

        if not is_qminus3.empty:
            is_qminus3_str = "\n".join(is_qminus3.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
            is_str += f"\n\nIncome Statement from three quarters ago:\n{is_qminus3_str}"
        if not cf_qminus3.empty:
            cf_qminus3_str = "\n".join(cf_qminus3.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))
            cf_str += f"\n\nCash Flow Statement from three quarters ago:\n{cf_qminus3_str}"
            

        # Call the LLM to get the rating
        response = client.models.generate_content(
           # model="gemini-2.5-flash", 
            model="gemini-2.5-flash-lite",
            config=types.GenerateContentConfig(
                temperature=0, # Deterministic ouput
                system_instruction="""You are an experienced, data-driven financial analyst, that provides concise and accurate answers.""",
                
                thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking, but only required for Gemini 2.5
            ),
            
            contents=[f"""
            Based on the following financial reports only, please provide an investment recommendation for the underlying company.
                      
            Balance Sheet: 
            {bs_str}

            Income Statement: 
            {is_str}

            Cash Flow Statement: 
            {cf_str}

            Provide your answer using only one of the following signals: 'buy', 'hold' or 'sell'.
            """]
        )

        # Extract rating from the response
        rating = response.text.strip().lower()
        llm_ratings.append({
            "cik": str(cik), 
            "date": date,
            "rating": rating
        })

    # If no ratings were generated, return None
    if not llm_ratings:
        return None
    
    # Convert the list of dictionaries to a DataFrame
    llm_ratings_df = pd.DataFrame(llm_ratings)

    llm_ratings_df["date"] = pd.to_datetime(llm_ratings_df["date"]).dt.date

    # Sort by Report Date
    llm_ratings_df.sort_values(by="date", inplace=True)

    # Reset index
    llm_ratings_df.reset_index(drop=True, inplace=True)
    
    return llm_ratings_df

In [ ]:
# Function to get LLM ratings for all CIKs
def llm_ratings_loop(
    cik_list,
    balance_sheets,
    income_statements,
    cash_flow_statements,
    function_to_use,
    output_path_ratings="../data/ciklist1_ratings_with_previous_quarters.csv",
    output_path_failed="../data/failed_ciks1.csv",
    retries=5,
    retry_delay=30
):
    """
    Process a list of CIKs to retrieve LLM ratings with retry logic on server errors.

    Args:
        cik_list (pd.DataFrame): DataFrame with a "CIK" column.
        balance_sheets (dict or DataFrame): Balance sheet data.
        income_statements (dict or DataFrame): Income statement data.
        cash_flow_statements (dict or DataFrame): Cash flow data.
        function_to_use (callable): Function to call for each CIK. (Either get_llm_ratings_with_previous_quarters or get_llm_ratings)
        output_path_ratings (str): File path to save ratings CSV.
        output_path_failed (str): File path to save failed CIKs.
        retries (int): Number of retry attempts on server error.
        retry_delay (int): Seconds to wait between retries.

    Returns:
        pd.DataFrame: Combined ratings DataFrame.
    """
    list_ratings = []
    failed_ciks = []
    progress_bar = tqdm(cik_list["CIK"], desc="Processing CIKs")

    for i, cik in enumerate(progress_bar):
        progress_bar.set_description(
            f"Processing CIK {i+1}/{len(cik_list)}: {cik} | Time: {pd.Timestamp.now().strftime('%H:%M:%S')}"
        )

        for attempt in range(retries):
            try:
                ratings = function_to_use(
                    cik,
                    balance_sheets,
                    income_statements,
                    cash_flow_statements
                )
                if ratings is not None:
                    list_ratings.append(ratings)
                break  # success, exit retry loop which starts at for attempt in range(retries)

            except (ServerError) as e:
                print(f"ServerError for CIK {cik} (Attempt {attempt + 1}/{retries}): {e}")
                if attempt < retries - 1:
                    time.sleep(retry_delay)
                else: # This else statement basically only runs if all retries failed i.e. the inner loop is completed, hence it starts with the next CIK
                    failed_ciks.append(cik)

    # Save results to CSV
    cik_ratings_df = pd.concat(list_ratings, ignore_index=True)
    cik_ratings_df.to_csv(output_path_ratings, index=False)

    # Save failed CIKs to CSV
    if failed_ciks:
        pd.Series(failed_ciks).to_csv(output_path_failed, index=False)

    return cik_ratings_df

- Finally, I can use this function to loop over all company CIKs 

In [6]:
# First determine unique CIKs
full_cik_set = set(labeled_balance_sheets["CIK"].unique()) | set(labeled_income_statements["CIK"].unique()) | set(labeled_cash_flow_statements["CIK"].unique())

# Convert to DataFrame to make slicing into equal parts easier
ciks = pd.DataFrame(full_cik_set, columns=["CIK"])

---

### Download of LLM ratings including financial data from previous quarters (going back up to 1 year, if possible)

In [ ]:
cik_list1 = ciks[:500]
cik_list2 = ciks[500:1000]
cik_list3 = ciks[1000:]
len(cik_list1) + len(cik_list2) + len(cik_list3) == len(ciks)

1491

In [ ]:
ratings1 = llm_ratings_loop(
    cik_list=cik_list1,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../results/ciklist1_ratings_with_previous_quarters.csv",
    output_path_failed="../results/failed_ciks1.csv",
    retries=5,
    retry_delay=30)

,CIK
0,0000080172
1,0000907242
2,0000745732
3,0000885725
4,0000793733


In [ ]:
ratings2 = llm_ratings_loop(
    cik_list=cik_list2,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../results/ciklist2_ratings_with_previous_quarters.csv",
    output_path_failed="../results/failed_ciks2.csv",
    retries=5,
    retry_delay=30)

Processing CIK 8/500: 0000949157 | Time: 11:32:08:   1%|▏         | 7/500 [05:49<7:21:43, 53.76s/it]

ServerError for CIK 0000949157 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 35/500: 0000040533 | Time: 11:56:55:   7%|▋         | 34/500 [30:35<6:31:01, 50.35s/it] 

ServerError for CIK 0000040533 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 65/500: 0000854775 | Time: 12:23:40:  13%|█▎        | 64/500 [57:21<5:16:49, 43.60s/it]

...

Processing CIK 418/500: 0000109198 | Time: 17:43:20:  83%|████████▎ | 417/500 [6:17:01<1:41:09, 73.13s/it]

ServerError for CIK 0000109198 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 495/500: 0000003453 | Time: 18:46:13:  99%|█████████▉| 494/500 [7:19:54<04:26, 44.34s/it]  

ServerError for CIK 0000003453 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 500/500: 0000887343 | Time: 18:51:03: 100%|██████████| 500/500 [7:25:44<00:00, 53.49s/it]


In [ ]:
ratings3 = llm_ratings_loop(
    cik_list=cik_list3,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../results/ciklist3_ratings_with_previous_quarters.csv",
    output_path_failed="../results/failed_ciks3.csv",
    retries=5,
    retry_delay=30)

Processing CIK 48/491: 0000058492 | Time: 14:42:36:  10%|▉         | 47/491 [36:03<5:50:44, 47.40s/it]

ServerError for CIK 0000058492 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 98/491: 0001519751 | Time: 15:24:16:  20%|█▉        | 97/491 [1:17:42<6:10:15, 56.38s/it]

ServerError for CIK 0001519751 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 116/491: 0000042582 | Time: 15:42:16:  23%|██▎       | 115/491 [1:35:43<6:20:24, 60.70s/it]

...

Processing CIK 177/491: 0000818479 | Time: 16:59:21:  36%|███▌      | 176/491 [2:52:48<4:58:01, 56.77s/it]

ServerError for CIK 0000818479 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 252/491: 0000005513 | Time: 17:54:04:  51%|█████     | 251/491 [3:47:31<2:05:28, 31.37s/it]

ServerError for CIK 0000005513 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 491/491: 0001792789 | Time: 20:43:24: 100%|██████████| 491/491 [6:37:02<00:00, 48.52s/it]  


- Downloading ratings for CIKs that were somehow missing

In [ ]:
# LLM recommendations with only most recent financial statements
llm_recommendations_with_previous_quarters1 = pd.read_csv("../results/ciklist1_ratings_with_previous_quarters.csv", dtype={"cik": str})
llm_recommendations_with_previous_quarters2 = pd.read_csv("../results/ciklist2_ratings_with_previous_quarters.csv", dtype={"cik": str})
llm_recommendations_with_previous_quarters3 = pd.read_csv("../results/ciklist3_ratings_with_previous_quarters.csv", dtype={"cik": str})

# Combine into one DataFrame
llm_recommendations_with_previous_quarters = pd.concat([llm_recommendations_with_previous_quarters1, llm_recommendations_with_previous_quarters2,
                                  llm_recommendations_with_previous_quarters3], ignore_index=True)

# Get missing CIKS
llm_ciks_with_previous_quarters = set(llm_recommendations_with_previous_quarters["cik"])
missing_ciks = full_cik_set - llm_ciks_with_previous_quarters
missing_ciks_list_with_previous_quarters = list(missing_ciks)
# Convert to df
missing_ciks_df_with_previous_quarters = pd.DataFrame(missing_ciks_list_with_previous_quarters, columns=["CIK"])
missing_ciks_df_with_previous_quarters

,CIK
0,0001175454
1,0001137774
2,0000744187
3,0001673985
4,0000007789
...,...
432,0000866729
433,0000043920
434,0001489393
435,0001413329


In [ ]:
missing_ratings_with_previous_quarters = llm_ratings_loop(
    cik_list=missing_ciks_df_with_previous_quarters,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    function_to_use=get_llm_ratings_with_previous_quarters,
    output_path_ratings="../results/new_llm_ratings/missing_ratings_with_previous_quarters.csv",
    output_path_failed="../results/new_llm_ratings/missing_failed_ciks_with_previous_quarters.csv",
    retries=5,
    retry_delay=30)

Processing CIK 1/437: 0001175454 | Time: 17:18:33:   0%|          | 0/437 [00:00<?, ?it/s]

Reports missing for date 2009-12-31T00:00:00.000000000: Balance Sheet. Skipping...
Reports missing for date 2010-03-31T00:00:00.000000000: Balance Sheet. Skipping...
Reports missing for date 2010-06-30T00:00:00.000000000: Balance Sheet. Skipping...
Reports missing for date 2010-09-30T00:00:00.000000000: Cash Flow Statement at Q0. Skipping...


Processing CIK 2/437: 0001137774 | Time: 17:19:08:   0%|          | 1/437 [00:34<4:13:29, 34.88s/it]

Reports missing for date 2000-12-31T00:00:00.000000000: Cash Flow Statement at Q0. Skipping...
Reports missing for date 2001-03-31T00:00:00.000000000: Balance Sheet. Skipping...
Reports missing for date 2001-06-30T00:00:00.000000000: Balance Sheet. Skipping...


Processing CIK 4/437: 0001673985 | Time: 17:21:03:   1%|          | 3/437 [02:30<6:19:31, 52.47s/it]

...

Processing CIK 435/437: 0001489393 | Time: 22:46:33:  99%|█████████▉| 434/437 [5:28:00<02:40, 53.63s/it]

Reports missing for date 2009-09-30T00:00:00.000000000: Balance Sheet. Skipping...
Reports missing for date 2009-12-31T00:00:00.000000000: Balance Sheet. Skipping...
Reports missing for date 2010-03-31T00:00:00.000000000: Balance Sheet. Skipping...


Processing CIK 436/437: 0001413329 | Time: 22:47:04: 100%|█████████▉| 435/437 [5:28:30<01:33, 46.59s/it]

Reports missing for date 2007-03-31T00:00:00.000000000: Balance Sheet. Skipping...
Reports missing for date 2007-06-30T00:00:00.000000000: Balance Sheet. Skipping...
Reports missing for date 2007-09-30T00:00:00.000000000: Balance Sheet. Skipping...
Reports missing for date 2007-12-31T00:00:00.000000000: Balance Sheet. Skipping...


Processing CIK 437/437: 0001020710 | Time: 22:47:38: 100%|██████████| 437/437 [5:29:56<00:00, 45.30s/it]


---
- Downloading recommendations with only most current financial statements

    - Using get_llm_ratings
    - Using the llm loop function 

In [ ]:
ratings1 = llm_ratings_loop(
    cik_list=cik_list1,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../results/ciklist1_ratings_with_most_recent_quarters.csv",
    output_path_failed="../results/failed_ciks1_most_recent_quarters.csv",
    retries=5,
    retry_delay=30)

Processing CIK 5/500: 0001368622 | Time: 13:10:25:   1%|          | 4/500 [03:07<6:46:34, 49.18s/it]

ServerError for CIK 0001368622 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}
ServerError for CIK 0001368622 (Attempt 2/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 10/500: 0000106640 | Time: 13:16:33:   2%|▏         | 9/500 [09:15<8:01:48, 58.88s/it]

ServerError for CIK 0000106640 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 14/500: 0000313927 | Time: 13:21:21:   3%|▎         | 13/500 [14:03<8:17:11, 61.25s/it] 

...

Processing CIK 493/500: 0001551306 | Time: 19:41:44:  98%|█████████▊| 492/500 [6:34:26<09:16, 69.58s/it]

ServerError for CIK 0001551306 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 498/500: 0000936468 | Time: 19:45:43:  99%|█████████▉| 497/500 [6:38:25<02:28, 49.54s/it]

ServerError for CIK 0000936468 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 500/500: 0000042582 | Time: 19:48:06: 100%|██████████| 500/500 [6:41:35<00:00, 48.19s/it]


In [ ]:
ratings2 = llm_ratings_loop(
    cik_list=cik_list2,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../results/ciklist2_ratings_with_most_recent_quarters.csv",
    output_path_failed="../results/failed_ciks2_most_recent_quarters.csv",
    retries=5,
    retry_delay=30)

Processing CIK 65/500: 0001235468 | Time: 10:16:44:  13%|█▎        | 64/500 [45:01<4:22:23, 36.11s/it]

ServerError for CIK 0001235468 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 227/500: 0000723188 | Time: 12:03:53:  45%|████▌     | 226/500 [2:32:11<3:42:18, 48.68s/it]

ServerError for CIK 0000723188 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 283/500: 0000046080 | Time: 12:42:48:  56%|█████▋    | 282/500 [3:11:06<2:25:58, 40.18s/it]

...

Processing CIK 470/500: 0000051253 | Time: 14:50:51:  94%|█████████▍| 469/500 [5:19:09<25:10, 48.71s/it]  

ServerError for CIK 0000051253 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 471/500: 0000875320 | Time: 14:52:50:  94%|█████████▍| 470/500 [5:21:08<34:55, 69.85s/it]

ServerError for CIK 0000875320 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 500/500: 0001013857 | Time: 15:17:13: 100%|██████████| 500/500 [5:46:28<00:00, 41.58s/it]


In [ ]:
ratings3 = llm_ratings_loop(
    cik_list=cik_list3,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    output_path_ratings="../results/ciklist3_ratings_with_most_recent_quarters.csv",
    output_path_failed="../results/failed_ciks3_most_recent_quarters.csv",
    retries=5,
    retry_delay=30)

Processing CIK 17/491: 0001309108 | Time: 15:31:52:   3%|▎         | 16/491 [13:41<5:42:15, 43.23s/it]

ServerError for CIK 0001309108 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 43/491: 0000801337 | Time: 15:53:52:   9%|▊         | 42/491 [35:41<5:24:34, 43.37s/it]

ServerError for CIK 0000801337 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 48/491: 0000049196 | Time: 15:58:35:  10%|▉         | 47/491 [40:24<5:45:57, 46.75s/it]

...

Processing CIK 467/491: 0000811596 | Time: 21:48:46:  95%|█████████▍| 466/491 [6:30:36<16:27, 39.50s/it]

ServerError for CIK 0000811596 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 478/491: 0000931015 | Time: 21:56:37:  97%|█████████▋| 477/491 [6:38:26<09:59, 42.81s/it]

ServerError for CIK 0000931015 (Attempt 1/5): 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}


Processing CIK 491/491: 0000818479 | Time: 22:05:57: 100%|██████████| 491/491 [6:48:37<00:00, 49.93s/it]


- Retrying CIKS that are somehow still missing in the final df

In [ ]:
# LLM recommendations with only most recent financial statements
llm_recommendations1 = pd.read_csv("../results/ciklist1_ratings_with_most_recent_quarters.csv", dtype={"cik": str})
llm_recommendations2 = pd.read_csv("../results/ciklist2_ratings_with_most_recent_quarters.csv", dtype={"cik": str})
llm_recommendations3 = pd.read_csv("../results/ciklist3_ratings_with_most_recent_quarters.csv", dtype={"cik": str})

# Combine into one DataFrame
llm_recommendations = pd.concat([llm_recommendations1, llm_recommendations2, llm_recommendations3], ignore_index=True)
llm_recommendations.head()

# Get missing CIKS
llm_ciks = set(llm_recommendations["cik"])
missing_ciks = full_cik_set - llm_ciks
missing_ciks_list_with_most_recent_quarters = list(missing_ciks)
# Convert to df
missing_ciks_df_with_most_recent_quarters = pd.DataFrame(missing_ciks_list_with_most_recent_quarters, columns=["CIK"])
missing_ciks_df_with_most_recent_quarters 

,CIK
0,0001673985
1,0001397187
2,0001398659
3,0001084048
4,0000718937
...,...
314,0001364250
315,0000866729
316,0001031296
317,0001636519


In [ ]:
missing_ratings_with_most_recent_quarters = llm_ratings_loop(
    cik_list=missing_ciks_df_with_most_recent_quarters,
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    function_to_use=get_llm_ratings,
    output_path_ratings="../results/new_llm_ratings/missing_ratings.csv", 
    output_path_failed="../results/new_llm_ratings/missing_failed_ciks.csv",
    retries=5,
    retry_delay=30)

Processing CIK 1/319: 0001673985 | Time: 12:52:44:   0%|          | 0/319 [00:00<?, ?it/s]

Reports missing for date 2015-09-30T00:00:00.000000000: Balance Sheet


Processing CIK 2/319: 0001397187 | Time: 12:53:01:   0%|          | 1/319 [00:17<1:34:48, 17.89s/it]

Reports missing for date 2005-04-30T00:00:00.000000000: Balance Sheet, Cash Flow Statement
Reports missing for date 2005-07-31T00:00:00.000000000: Balance Sheet, Cash Flow Statement
Reports missing for date 2005-10-31T00:00:00.000000000: Balance Sheet, Cash Flow Statement
Reports missing for date 2006-01-31T00:00:00.000000000: Balance Sheet, Cash Flow Statement
Reports missing for date 2006-07-31T00:00:00.000000000: Balance Sheet
Reports missing for date 2006-10-31T00:00:00.000000000: Balance Sheet
Reports missing for date 2007-01-31T00:00:00.000000000: Balance Sheet
Reports missing for date 2008-05-04T00:00:00.000000000: Balance Sheet
Reports missing for date 2008-08-03T00:00:00.000000000: Balance Sheet, Cash Flow Statement
Reports missing for date 2008-11-02T00:00:00.000000000: Cash Flow Statement
Reports missing for date 2010-01-31T00:00:00.000000000: Balance Sheet
Reports missing for date 2011-10-30T00:00:00.000000000: Cash Flow Statement


Processing CIK 3/319: 0001398659 | Time: 12:53:31:   1%|          | 2/319 [00:47<2:12:16, 25.04s/it]

...

Processing CIK 318/319: 0001636519 | Time: 17:00:32:  99%|█████████▉| 317/319 [4:07:48<01:42, 51.27s/it]

Reports missing for date 2014-03-31T00:00:00.000000000: Balance Sheet
Reports missing for date 2014-06-30T00:00:00.000000000: Balance Sheet
Reports missing for date 2014-09-30T00:00:00.000000000: Balance Sheet
Reports missing for date 2014-12-31T00:00:00.000000000: Cash Flow Statement


Processing CIK 319/319: 0001822993 | Time: 17:00:53: 100%|█████████▉| 318/319 [4:08:09<00:41, 41.94s/it]

Reports missing for date 2020-06-30T00:00:00.000000000: Balance Sheet
Reports missing for date 2020-09-30T00:00:00.000000000: Balance Sheet
Reports missing for date 2020-12-31T00:00:00.000000000: Balance Sheet


Processing CIK 319/319: 0001822993 | Time: 17:00:53: 100%|██████████| 319/319 [4:08:16<00:00, 46.70s/it]


---

### Prompting Gemini to produce a signal and reasoning for its choice

The function used for this, is taken from the gemini_inferring.ipynb notebook and slightly adjusted to fit the task at hand. 
We will prompt the LLM 25.000 times, picking a random CIK and date each time. 

In [1]:
def get_llm_ratings_with_reasons(cik: str, balance_sheets = None, income_statements = None, cash_flow_statements = None):
    """
    Function that queries Gemini to provide a buy/hold/sell recommendation along with reasons for a given CIK.
    For a randomly drawn date, the function fetches the most recent financial statements, i.e.
    - Balance Sheet
    - Cash Flow Statement
    - Income Statement
    and calls the LLM to get a buy/sell/hold recommendation along with reasons for that choice. In order to avoid issues with reports that were filed slightly apart,
    a window of 10 days around a given reporting date is used. This helps to ensure that the LLM has access to all relevant financial information for a given reporting date.
    Furthermore, reports that were filed slightly apart will not lead to recommendations that are based on partial information only and will also not cause multiple
    recommendations that only lie within the window of 10 days around a given reporting date.

    Parameters: 
    cik: str, CIK of the company (Can be looked up on the SEC website)
    balance_sheets: Balance Sheets DataFrame with columns: STD Balance Sheet All, FCC Item Name, CIK and Date (among others)
    income_statements: Income Statements DataFrame with columns: STD Income Statement All, FCC Item Name, CIK and Date (among others)
    cash_flow_statements: Cash Flow Statements DataFrame with columns: STD Cash Flow All, FCC Item Name, CIK and Date (among others)
    """
    
    # First filter dfs for input CIK
    balance_sheets = balance_sheets[balance_sheets["CIK"] == cik].copy()
    income_statements = income_statements[income_statements["CIK"] == cik].copy()
    cash_flow_statements = cash_flow_statements[cash_flow_statements["CIK"] == cik].copy()
    
    # Convert the date columns to datetime objects
    for df in [balance_sheets, income_statements, cash_flow_statements]:
        df["Report Date"] = pd.to_datetime(df["Date"])

    # Determine unique dates
    reporting_dates = pd.concat([
        balance_sheets["Report Date"],
        income_statements["Report Date"],
        cash_flow_statements["Report Date"]
    ]).unique()
    
    # In order to handle reports, that were filed slighty apart, a window of 10 days around a given reporting date is used
    window = pd.Timedelta(days=10)
    
    # Sort dates
    reporting_dates = np.sort(reporting_dates)
    # Create empty set to track already attempted dates
    attempted_dates = set()

    # Randomly sample reporting dates until one is found, for which all three reports are available
    while len(attempted_dates) < len(reporting_dates):
        date = pd.to_datetime(np.random.choice(reporting_dates, size=1)[0])
        if date in attempted_dates:
            continue # continues at top of while loop
        attempted_dates.add(date)
        # Filter reports for the given date +/- window
        bs = balance_sheets[(balance_sheets["Report Date"] >= date - window) &
                            (balance_sheets["Report Date"] <= date + window)]
        is_0 = income_statements[(income_statements["Report Date"] >= date - window) &
                                (income_statements["Report Date"] <= date + window)]
        cf_0 = cash_flow_statements[(cash_flow_statements["Report Date"] >= date - window) &
                                    (cash_flow_statements["Report Date"] <= date + window)]
        # If all three reports are non-empty, we can proceed
        if not bs.empty and not is_0.empty and not cf_0.empty:
            break  # valid date found, continue at next line after while loop

    else: # else clause belongs to while loop, only executed if while loop is not broken out of via 'break'
        print(f"No valid reporting date found for CIK {cik}. Skipping...")
        return None
    
    # Concatenate reports into strings with correct labels
    bs_str = "\n".join(bs.apply(lambda row: f"{row['position_label']}: {row['STD Balance Sheet All']}", axis=1).astype(str))
    is_str = "\n".join(is_0.apply(lambda row: f"{row['position_label']}: {row['STD Income Statement All']}", axis=1).astype(str))
    cf_str = "\n".join(cf_0.apply(lambda row: f"{row['position_label']}: {row['STD Cash Flow All']}", axis=1).astype(str))        



    # Call the LLM to get the rating
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite", # "gemini-2.5-flash"
        config=types.GenerateContentConfig(
            temperature=0, # Deterministic ouput
            system_instruction="""You are an experienced, data-driven financial analyst, that provides concise and accurate answers.""",
            
            thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking, but only required for Gemini 2.5
        ),
        
        contents=[f"""
        Based on the following financial reports only, please provide an investment recommendation for the underlying company along with a very concise list of reasons for that choice.
                    
        Balance Sheet: 
        {bs_str}

        Income Statement: 
        {is_str}

        Cash Flow Statement: 
        {cf_str}

        Please provide your answer in the following dictionary style format:
        {{
            "signal": <buy/hold/sell>,
            "reason": <Reason 1, Reason 2, Reason 3, Reason 4, Reason 5>,
        }}

        It is imperative to follow the above format and give only one of the following signals: 'buy', 'hold' or 'sell' and concise reasons in a list format.
            """]
    )

    # Extract rating from the response
    result = response.text.strip()
    # Remove Markdown fences if present (?)
    result = re.sub(r"^```json\s*|\s*```$", "", result, flags=re.DOTALL).strip()

    # Try to parse the result as a dictionary
    try:
        parsed = json.loads(result)
    except json.JSONDecodeError:
        print(f"Could not parse result for CIK {cik} on {date}. Raw output: {result}")
        parsed = {}
    
    # Append results to list
    result_dict = {
        "CIK": cik,
        "date": pd.to_datetime(date).to_period("M"),
        "signal": parsed.get("signal", None),
        "reason": parsed.get("reason", None)
    }

    return result_dict

In [2]:
# Function to loop over a list of CIKs and apply the gemini_infer functions
def gemini_infer_loop(cik_list, balance_sheets, income_statements, cash_flow_statements, function_to_use, no_requests = 10000):
    results = []
    for request in tqdm(range(no_requests)):
        # Pick random CIK
        cik = np.random.choice(cik_list, size=1)[0]
        success = False
        attempts = 0
        while not success and attempts < 5:  # Retry up to 5 times
            try:
                result = function_to_use(
                    cik=cik,
                    balance_sheets=balance_sheets,
                    income_statements=income_statements,
                    cash_flow_statements=cash_flow_statements
                )
                if result is not None:
                    results.append(result)
                success = True
            except ServerError as e:
                attempts += 1
                print(f"ServerError encountered for CIK {cik}. Attempt {attempts}/5. Error: {e}")
                time.sleep(5)  # Wait before retrying
            except Exception as e:
                print(f"Unexpected error for CIK {cik}: {e}")
                success = True  # Exit loop on unexpected errors
    return pd.DataFrame(results)

In [9]:
llm_reasoning_results = gemini_infer_loop(
    cik_list=ciks["CIK"],
    balance_sheets=labeled_balance_sheets,
    income_statements=labeled_income_statements,
    cash_flow_statements=labeled_cash_flow_statements,
    function_to_use=get_llm_ratings_with_reasons,
    no_requests=20000
)
# Save to CSV
llm_reasoning_results.to_csv("../results/llm_recommendations_with_reasons.csv", index=False)

  1%|          | 197/20000 [06:02<9:14:47,  1.68s/it] 

No valid reporting date found for CIK 0001996810. Skipping...


  2%|▏         | 334/20000 [10:16<8:40:29,  1.59s/it] 

No valid reporting date found for CIK 0002027317. Skipping...


  4%|▍         | 846/20000 [27:08<8:54:13,  1.67s/it]  

...

 90%|█████████ | 18023/20000 [9:09:51<53:01,  1.61s/it]  

No valid reporting date found for CIK 0002015845. Skipping...


 91%|█████████ | 18105/20000 [9:12:19<51:48,  1.64s/it]  

No valid reporting date found for CIK 0002015845. Skipping...


100%|██████████| 20000/20000 [10:10:21<00:00,  1.83s/it] 


In [16]:
llm_reasoning_results

,CIK,date,signal,reason
0,0001597033,2023-12,sell,[Significant negative net income and comprehen...
1,0000922224,2017-09,hold,"[Positive operating cash flow generation., Sig..."
2,0001428205,2022-03,sell,[Significant net loss reported in the income s...
3,0001418135,2021-03,buy,"[Strong positive net income of $325 million., ..."
4,0001551152,2022-03,hold,[Significant intangible assets and goodwill re...
...,...,...,...,...
19954,0000921082,2012-06,hold,[Significant investment property holdings sugg...
19955,0001164727,2021-09,hold,"[Positive operating cash flow generation., Sig..."
19956,0001506307,2021-06,sell,[Significant net loss reported in the income s...
19957,0001433195,2018-06,buy,"[Strong positive net income and EPS growth., S..."


---

### Using financial ratios derived from raw financial statements

In [ ]:
llm_df = pd.read_csv("../results/llm_recommendations_with_financial_metrics.csv", dtype={"cik": str})
ciklist = llm_df["cik"].unique().tolist()

Deleted price to earnings for now, as it is a huge outlier in itself

In [5]:
def get_llm_ratings_with_reasons_from_ratios(cik: str, recommendations_df: pd.DataFrame):
    """
    Function that generates and returns the *prompt content* that would be 
    sent to the LLM. This is a check for you to see what you are feeding into the model.
    
    For a randomly drawn date, the function fetches the pre-computed financial ratios
    for that CIK and date, and formats them into the final prompt string.

    Parameters: 
    cik: str, CIK of the company
    recommendations_df: DataFrame containing pre-computed financial metrics, 
                        with columns including 'cik', 'date', and all ratio names.
    """
    
    # First filter df for input CIK
    recommendations_df = recommendations_df[recommendations_df["cik"] == cik].copy()

    if recommendations_df.empty:
        print(f"No records found for CIK {cik}.")
        return None
    
    # Convert the date columns to datetime objects
    recommendations_df["date"] = pd.to_datetime(recommendations_df["date"])

    # Determine unique dates
    reporting_dates = recommendations_df["date"].unique()

    # Subset random date 
    date = pd.to_datetime(np.random.choice(reporting_dates, size=1)[0])

    # Filter records for the given date
    recs = recommendations_df[recommendations_df["date"] == date]   

    # Construct String representation of financial metrics
    metrics_string = f"""
    Book to Market: {recs["book_to_market"].values[0]} \n
    Intereset Coverage: {recs["interest_coverage"].values[0]} \n
    Market Capitalization: {recs["market_capitalization"].values[0]} \n
    Cash Flow to Price: {recs["cash_flow_to_price"].values[0]} \n
    Return on Equity: {recs["return_on_equity"].values[0]} \n
    Earnings to Price: {recs["earnings_to_price"].values[0]} \n
    Working Capital to total Assets: {recs["working_capital_to_total_assets"].values[0]} \n
    Retained Earnings to total Assets: {recs["retained_earnings_to_total_assets"].values[0]} \n
    EBIT to total Assets: {recs["ebit_to_total_assets"].values[0]} \n
    Market Cap to total Liablities: {recs["market_cap_to_total_liabilities"].values[0]} \n
    Sales to Assets: {recs["sales_to_assets"].values[0]} \n
    Operating Margin: {recs["operating_margin"].values[0]} \n
    Debt to Equity: {recs["debt_to_equity"].values[0]} \n
    Debt to Assets: {recs["debt_to_assets"].values[0]} \n
    """

    # Build the prompt
    prompt_content = f"""
    Based on the following financial metrics only, please provide an investment recommendation for the underlying company along with a very concise list of reasons for that choice.
    Financial Metrics: 
    {metrics_string}
    Please provide your answer in the following dictionary style format:
    {{
        "signal": <buy/hold/sell>,
        "reason": <Reason 1, Reason 2, Reason 3, Reason 4, Reason 5>,
    }}
    As Reasons, simply list the names of the most important financial metrics that influenced your decision.
    It is imperative to follow the above format and give only one of the following signals: 'buy', 'hold' or 'sell' and concise reasons in a list format.
    """

    # Call the LLM to get the rating
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite", # "gemini-2.5-flash"
        config=types.GenerateContentConfig(
            temperature=0, # Deterministic ouput
            system_instruction="""You are an experienced, data-driven financial analyst, that provides concise and accurate answers.""",
            
            thinking_config=types.ThinkingConfig(thinking_budget=0),# Disables thinking, but only required for Gemini 2.5
        ),
        contents=[prompt_content]
    )

    # Extract rating from the response
    result = response.text.strip()
    # Remove Markdown fences if present (?)
    result = re.sub(r"^```json\s*|\s*```$", "", result, flags=re.DOTALL).strip()

    # Try to parse the result as a dictionary
    try:
        parsed = json.loads(result)
    except json.JSONDecodeError:
        print(f"Could not parse result for CIK {cik} on {date}. Raw output: {result}")
        parsed = {}
    
    # Append results to list
    result_dict = {
        "CIK": cik,
        "date": pd.to_datetime(date).to_period("M"),
        "signal": parsed.get("signal", None),
        "reason": parsed.get("reason", None)
    }

    return result_dict

In [6]:
def metrics_infer_loop(cik_list, recommendations_df, no_requests = 10000):
    requests = []
    for request in tqdm(range(no_requests)):
        # Pick random CIK
        cik = np.random.choice(cik_list, size=1)[0]
        success = False
        attempts = 0
        while not success and attempts < 5:  # Retry up to 5 times
            try:
                result = get_llm_ratings_with_reasons_from_ratios(
                    cik=cik,
                    recommendations_df=recommendations_df
                )
                if result is not None:
                    requests.append(result)
                success = True
            except ServerError as e:
                attempts += 1
                print(f"ServerError encountered for CIK {cik}. Attempt {attempts}/5. Error: {e}")
                time.sleep(5)  # Wait before retrying
            except Exception as e:
                print(f"Unexpected error for CIK {cik}: {e}")
                success = True  # Exit loop on unexpected errors
    return pd.DataFrame(requests)

In [10]:
metrics_ratings_df = metrics_infer_loop(
    cik_list=ciklist,
    recommendations_df=llm_df,
    no_requests = 20000
)
# Save to CSV
metrics_ratings_df.to_csv("../results/llm_recommendations_with_reasons_from_financial_metrics2.csv", index=False)

100%|██████████| 20000/20000 [2:51:41<00:00,  1.94it/s]  


In [11]:
metrics_ratings_df["signal"].value_counts()

signal
sell    12225
hold     6144
buy      1631
Name: count, dtype: int64